<a href="https://colab.research.google.com/github/AraiKensuke/LOST/blob/Upgrade_python3/FromLocalFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#########  clone and install necessary packages from github repositories

import sys
import importlib

if importlib.util.find_spec("pyPG") is None:
  !git clone https://github.com/AraiKensuke/pyPG.git
  %cd /content/pyPG/
   ###  pyPG is written in C++/python, and uses the C library GNU Scientific Library.  
  !sudo apt-get install libgsl-dev    
  !python setup.py build_ext --inplace




Cloning into 'pyPG'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.
/content/pyPG
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgsl23 libgslcblas0
Suggested packages:
  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html
The following NEW packages will be installed:
  libgsl-dev libgsl23 libgslcblas0
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 1,926 kB of archives.
After this operation, 9,474 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgslcblas0 amd64 2.4+dfsg-6 [79.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl23 amd64 2.4+dfsg-6 [823 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl-dev amd64 2.4+dfsg-6 [1,023 kB]
Fetched 1,926 kB in 2s 

In [2]:
if importlib.util.find_spec("LOST") is None:
  %cd /content
  !git clone https://github.com/AraiKensuke/LOST.git
  %cd /content/LOST

  !python setup.py build_ext --inplace

sys.path.insert(1, "/content/LOST")    #  add these to searchpath for python modules
sys.path.insert(1, "/content/pyPG")    #  add these to searchpath for python modules


#from google.colab import files
#uploaded = files.upload()

/content
Cloning into 'LOST'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 196 (delta 18), reused 31 (delta 10), pack-reused 141
Receiving objects: 100% (196/196), 952.58 KiB | 3.51 MiB/s, done.
Resolving deltas: 100% (89/89), done.
/content/LOST
running build_ext
cythoning kfARlibMPmv_ram2.pyx to kfARlibMPmv_ram2.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/LOST/kfARlibMPmv_ram2.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'kfARlibMPmv_ram2' extension
creating build
creating build/temp.linux-x86_64-3.6
gcc -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/co

In [4]:
from google.colab import files
uploaded = files.upload()

keys = list(uploaded.keys())

os.mkdir("%(dir)s/%(fn)s" % {"dir" : __exec_base_fn__, "fn" : keys[0]})

/content/LOST
/content


In [23]:
"""
Template to invoke Latent Oscillation in Spike Train (LOST) model

There are 
"""

import numpy as _N
import os


import LOST
#from LOST.tmrsclTest import timeRescaleTest, zoom
#from LOST.mcmcFigs import getComponents, plotWFandSpks, showFsHist, filterByFsAmps, plotPSTH, corrcoeffs, plotFsAmp, acorrs
from LOST.ARcfSmplFuncs import dcmpcff
import LOST.mcmcARp_ram as _mARp
#import mcmcARp as _mARp
import inspect
import __main__ as main
import scipy.stats as _ss
#import LOST.mcmcFigs as mF
import pickle
import parseFN as pF
import LOST.utilities as _U


###################  params and settings
bRealDat         = False      #  is our data real data <lfp> <fltrd> <spk 0 1>
pkldSmpls        = "smplsN.dump"   #  last Gibbs samples, pickled for continuation
postModes        = None       #  MAP estimate of parameters
dontrun          = False      #  just construct mARp from pickle

###################  params and settings
bRealDat         = False      #  is our data real data <lfp> <fltrd> <spk 0 1>
pkldSmpls        = "smplsN.dump"   #  last Gibbs samples, pickled for continuation
postModes        = None       #  MAP estimate of parameters
dontrun          = False      #  just construct mARp from pickle

__exec_base_fn__ = "mcmcARp_demo"
nwpr = "wp"
tr0              = 0
tr1              = 40
Cn               = 5
R                = 1

if not os.access(__exec_base_fn__, os.F_OK):
    os.mkdir(__exec_base_fn__)


mARp = _mARp.mcmcARp()
if nwpr == "wp":
    mARp.sig_ph0H =-(0.97*0.97)

mARp.mcmcRunDir= __exec_base_fn__
mARp.burn     = 1500
mARp.NMC      = 0
mARp.peek     = 200
mARp.model    = "bernoulli"
mARp.t0       = 0
mARp.t1       = 1200
mARp.ID_q2    = False
mARp.bpsth    = True
mARp.dt       = 0.001
mARp.histFN   = ""
mARp.ifs      = [(0.3 / mARp.fSigMax) * _N.pi]    #  initial values
mARp.Cn       = Cn
mARp.R        = R
mARp.knownSigFN= None
mARp.xknownSig=0.9
mARp.BsmpxSkp = 10
mARp.doBsmpx  = True
mARp.noAR     = False
mARp.processes= 1
latentITERS   = 200   #  when doing runLatent





In [22]:
!ls

LOST  mcmcARp_demo  pyPG  sample_data


In [ ]:
mARp.burn = latentITERS
mARp.NMC  = 0
mARp.run(runDir=__exec_base_fn__, trials=_N.arange(tr0, tr1), pckl=pckl, runlatent=True, dontrun=dontrun)


In [24]:
from google.colab import files
uploaded = files.upload()

Saving xprbsdN.dat to xprbsdN.dat


In [ ]:
uploaded

In [27]:
keys = uploaded.keys()

In [28]:
keys[0]

TypeError: ignored

In [30]:
list(keys)[0]

'xprbsdN.dat'

In [31]:
keys = list(uploaded.keys())

os.mkdir("%(dir)s/%(fn)s" % {"dir" : __exec_base_fn__, "fn" : keys[0]})

In [32]:
!ls

LOST  mcmcARp_demo  pyPG  sample_data  xprbsdN.dat


In [33]:
__exec_base_fn__

'mcmcARp_demo'

In [34]:
!ls mcmcARp_demo

xprbsdN.dat
